In [ ]:
import cv2
import numpy as np
import onnxruntime as ort

In [ ]:
def preprocess_image(img: np.ndarray, input_size: tuple[int,int] = (112, 112)) -> np.ndarray:
    """
    Preprocess image for LVFace inference
    Args:
        img (np.ndarray): Input image in BGR format (from cv2.imread)
    Returns:
        np.ndarray: Preprocessed image tensor
    """
    # Resize image to input size
    img_resized = cv2.resize(img, input_size)
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    # Transpose to (C, H, W)
    img_transposed = np.transpose(img_rgb, (2, 0, 1))
    # Normalize to [-1, 1]
    img_normalized = ((img_transposed / 255.0) - 0.5) / 0.5
    # Convert to float32 and add batch dimension
    img_tensor = img_normalized.astype(np.float32)[np.newaxis, ...]
    return img_tensor

def create_session(model_path: str):
    providers = [
        'CUDAExecutionProvider',
        'OpenVINOExecutionProvider',
        'TensorrtExecutionProvider',
        'CPUExecutionProvider',
    ]
    session = ort.InferenceSession(
        model_path,
        providers=providers,
        provider_options=None
    )
    print(f"[INFO] ONNX provider: {session.get_providers()[0]}")
    return session

In [ ]:
onnx_path = "LVFace-S_Glint360K.onnx"
img_list = [
            cv2.imread('../example-data/person1_1.jpeg'),
            cv2.imread('../example-data/person1_2.jpg'),
            cv2.imread('../example-data/person2_1.jpg'),
            cv2.imread('../example-data/person2_2.jpg'),]

In [ ]:
detections = [[{'bbox': [117, 79, 309, 372],
   'keypoints': [171, 208, 263, 205, 220, 268, 183, 306, 256, 303],
   'conf': 0.9086456298828125}],
 [{'bbox': [207, 48, 398, 349],
   'keypoints': [254, 175, 345, 174, 299, 234, 265, 277, 338, 277],
   'conf': 0.8973720073699951}],
 [{'bbox': [186, 191, 545, 691],
   'keypoints': [284, 413, 443, 411, 364, 520, 307, 583, 428, 580],
   'conf': 0.9050660133361816}],
 [{'bbox': [83, 34, 285, 325],
   'keypoints': [141, 154, 230, 153, 186, 203, 147, 244, 225, 242],
   'conf': 0.9086183309555054}],
 [{'bbox': [122, 103, 372, 452],
   'keypoints': [158, 243, 265, 239, 192, 325, 171, 350, 286, 346],
   'conf': 0.8931349515914917}],
 [{'bbox': [495, 217, 806, 644],
   'keypoints': [545, 411, 677, 381, 607, 491, 589, 552, 705, 527],
   'conf': 0.9142543077468872}]]

In [ ]:
TEMPLATE_112 = np.array([
    [38.2946, 51.6963],
    [73.5318, 51.5014],
    [56.0252, 71.7366],
    [41.5493, 92.3655],
    [70.7299, 92.2041]
], dtype=np.float32)

def extract_largest_face_aligned(img: np.ndarray, detections: list[dict], output_size: int = 112) -> np.ndarray | None:
    if not detections:
        return None

    areas = [(d['bbox'][2]-d['bbox'][0]) * (d['bbox'][3]-d['bbox'][1]) for d in detections]
    best = detections[np.argmax(areas)]
    kps = np.array(best['keypoints']).reshape(5, 2).astype(np.float32)

    M, _ = cv2.estimateAffinePartial2D(kps, TEMPLATE_112, method=cv2.LMEDS)
    if M is None:
        return None

    aligned = cv2.warpAffine(
        img,
        M,
        (output_size, output_size),
        borderMode=cv2.BORDER_REPLICATE,
        flags=cv2.INTER_LINEAR
    )
    return aligned

faces = []
for i, img in enumerate(img_list):
    face = extract_largest_face_aligned(img, detections[i])
    if face is not None:
        faces.append(face)
    else:
        print(f"No faces on image {i}")

In [ ]:
tensor_list = []
for face in faces:
    face_tensor = preprocess_image(face)
    tensor_list.append(face_tensor)
input = np.vstack(tensor_list)

session = create_session("../embedding_service/LVFace-S_Glint360K.onnx")

output = session.run(
    None,
    {"data": input}
)

In [ ]:
input[0].mean()

In [ ]:
def calculate_similarity(feat1: np.ndarray, feat2: np.ndarray) -> float:
    """
    Calculate cosine similarity between two features
    
    Args:
        feat1 (np.ndarray): First feature embedding
        feat2 (np.ndarray): Second feature embedding
        
    Returns:
        float: Cosine similarity score (range: [-1, 1])
    """
    # Flatten features
    feat1_flat = np.ravel(feat1)
    feat2_flat = np.ravel(feat2)
    
    # Calculate cosine similarity
    dot_product = np.dot(feat1_flat, feat2_flat)
    norm1 = np.linalg.norm(feat1_flat)
    norm2 = np.linalg.norm(feat2_flat)
    
    return dot_product / (norm1 * norm2) if (norm1 > 0 and norm2 > 0) else 0.0

print("1_1 1_2: ", calculate_similarity(output[0][0], output[0][1]))
print("1_1 2_1: ", calculate_similarity(output[0][0], output[0][2]))
print("1_2 2_2: ", calculate_similarity(output[0][1], output[0][3]))
print("2_1 2_2: ", calculate_similarity(output[0][2], output[0][3]))